In [ ]:
#EVERYTHING IN HERE IS LEGACY, TURNS OUT HERE WAS A SOLAR PROTON EVENT LIST ON THE NOAA WEBSITE, KEEPING THIS HERE FOR A MOMENT


import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import data_loader
import date_helper
import datetime
import numpy as np
import matplotlib.pyplot as plt
from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import pandas as pd
import spacepy.time

import importlib
importlib.reload(data_loader)
importlib.reload(date_helper)

%matplotlib qt

In [3]:
def replace_nans_with_previous_valid_data(data):
    
    is_nan = np.nonzero(np.isnan(data))[0]
            
    for nan in is_nan:
        
        i = nan
        j = 1
        
        while True:
            
            if (i - j) == -1:
                break
            
            elif np.isfinite(data[i - j]):
                
                data[nan] = data[i - j]
                break
            
            j += 1
            
    return data

In [4]:
def find_contaminated_periods(dates, flux):
    
    flux = replace_nans_with_previous_valid_data(flux)
        
    #This removes any NaNs at the beginning that weren't able to be filled with previous values
    is_finite = np.isfinite(flux)
    dates = dates[is_finite]
    flux = flux[is_finite]
            
    start_pts = []
    end_pts = []
    
    if flux[0] >= 10:
        start_pts.append(0)
            
    a = np.append(np.nan, flux)
    b = np.append(flux, np.nan)
    
    crosses_ten_increasing = np.nonzero((a < 10) & (b >= 10))[0]-1
    start_pts.extend(crosses_ten_increasing)
    
    crosses_ten_decreasing = np.nonzero((a >= 10) & (b < 10))[0]
    end_pts.extend(crosses_ten_decreasing)
    
    if flux[len(flux) - 1] >= 10:
        end_pts.append(len(flux) - 1)
            
    start_dates = dates[start_pts]
    end_dates = dates[end_pts]        
    
    return [(start_dates[i], end_dates[i]) for i in range(len(start_dates))]

In [38]:
def integrate_differential_flux(diff_proton_flux, int_proton_flux_over_500_MeV, diff_proton_lower_energy, diff_proton_upper_energy, diff_proton_effective_energy, timestamps_per_file):
    
    diff_proton_flux[diff_proton_flux < 0] = np.nan
    int_proton_flux_over_500_MeV[int_proton_flux_over_500_MeV < 0] = np.nan
    
    
    integrated_flux = []
    num_files = diff_proton_lower_energy.shape[0]
    index = 0
    for f in range(num_files):
        
        lower_energies = np.squeeze(diff_proton_lower_energy[f, :, :])
        upper_energies = np.squeeze(diff_proton_upper_energy[f, :, :])
        effective_energies = np.squeeze(diff_proton_effective_energy[f, :, :])
        
        sensor_1_lower_energies = lower_energies[0, :]
        sensor_2_lower_energies = lower_energies[1, :]
        sensor_1_upper_energies = upper_energies[0, :]
        sensor_2_upper_energies = upper_energies[1, :]
        sensor_1_effective_energies = effective_energies[0, :]
        sensor_2_effective_energies = effective_energies[1, :]        
        
        sensor_1_energies_above_10_MeV = (10.0 <= sensor_1_effective_energies)
        sensor_2_energies_above_10_MeV = (10.0 <= sensor_2_effective_energies)
                
        sensor_1_dE = sensor_1_upper_energies[sensor_1_energies_above_10_MeV] - sensor_1_lower_energies[sensor_1_energies_above_10_MeV]
        sensor_2_dE = sensor_2_upper_energies[sensor_2_energies_above_10_MeV] - sensor_2_lower_energies[sensor_2_energies_above_10_MeV]
        sensor_1_dE = np.expand_dims(sensor_1_dE, axis=0)
        sensor_2_dE = np.expand_dims(sensor_2_dE, axis=0)
                        
        sensor_1_diff_fluxes = diff_proton_flux[index : (index + timestamps_per_file[f]), 0, :][:, sensor_1_energies_above_10_MeV]
        sensor_2_diff_fluxes = diff_proton_flux[index : (index + timestamps_per_file[f]), 1, :][:, sensor_2_energies_above_10_MeV]
        
        sensor_1_integral_flux_above_10_MeV = np.nansum(sensor_1_diff_fluxes * sensor_1_dE, axis=1) + int_proton_flux_over_500_MeV[index : (index + timestamps_per_file[f]), 0]
        sensor_2_integral_flux_above_10_MeV = np.nansum(sensor_2_diff_fluxes * sensor_2_dE, axis=1) + int_proton_flux_over_500_MeV[index : (index + timestamps_per_file[f]), 1]
                
                
        
        integrated_flux.extend(np.nanmean([sensor_1_integral_flux_above_10_MeV, sensor_2_integral_flux_above_10_MeV], axis=0))
        
        index += timestamps_per_file[f]
        
    
    return integrated_flux

In [39]:
# Need to load data, clean it, then save it in some sort of numpy array somewhere
        
periods_to_remove : list[tuple] = [] 
start_of_study = datetime.datetime(year = 1998, month=1, day=1)
end_of_study = datetime.datetime(year = 2024, month=12, day=31, hour=23, minute=59, second=59)

#We left g12 out because all of the data in p3_flux_ic is nan!
for satellite in ["g08", "g09", "g10", "g11", "g13", "g14", "g15", "g16", "g17", "g18"]:
    
    print(satellite)
    
    if satellite in ["g08", "g09", "g10", "g11"]:
        
        continue
        EPS = data_loader.load_raw_data_from_config(id = ["GOES", "AVG", "EPS"], 
                                                    start = start_of_study,
                                                    end = end_of_study,
                                                    satellite = satellite)
        
        if not EPS:
            continue
        
        time_tag = EPS["time_tag"]
        flux = EPS["p3_flux_ic"]
        flux[flux < 0] = np.nan
        
        good_time_tags = (time_tag > 0)
        time_tag = time_tag[good_time_tags]
        flux = flux[good_time_tags]
        
        dates = pd.to_datetime(EPS["time_tag"] / 1000, origin="unix", unit="s").to_pydatetime()
                
        during_study = (start_of_study <= dates) & (dates <= end_of_study)
        dates = dates[during_study]
        flux = flux[during_study]
        
        periods_to_remove.extend(find_contaminated_periods(dates, flux))
        
                    
    if satellite in ["g13", "g14", "g15"]:
        continue
        EPEAD = data_loader.load_raw_data_from_config(id = ["GOES", "AVG", "EPEAD"], 
                                                        start = start_of_study,
                                                        end = end_of_study,
                                                        satellite = satellite)
        
        if not EPEAD:
            continue
        
        time_tag = EPEAD["time_tag"]
        flux_east = EPEAD["ZPGT10E"]
        flux_west = EPEAD["ZPGT10W"]
        flux = (flux_east + flux_west) / 2
        flux[flux < 0] = np.nan
        
        good_time_tags = (time_tag > 0)
        time_tag = time_tag[good_time_tags]
        flux = flux[good_time_tags]
        
        dates = pd.to_datetime(EPEAD["time_tag"] / 1000, origin="unix", unit="s").to_pydatetime()
        
        during_study = (start_of_study <= dates) & (dates <= end_of_study)
        dates = dates[during_study]
        flux = flux[during_study]
        
        periods_to_remove.extend(find_contaminated_periods(dates, flux))
        
        
    if satellite in ["g18"]:
        
        SGPS_v1 = data_loader.load_raw_data_from_config(id = ["GOES", "AVG", "SGPS", "V1"], 
                                                        start = start_of_study,
                                                        end = end_of_study,
                                                        satellite = satellite)
        SGPS_v2 = data_loader.load_raw_data_from_config(id = ["GOES", "AVG", "SGPS", "V2"], 
                                                        start = start_of_study,
                                                        end = end_of_study,
                                                        satellite = satellite)  
        SGPS_v3 = data_loader.load_raw_data_from_config(id = ["GOES", "AVG", "SGPS", "V3"], 
                                                        start = start_of_study,
                                                        end = end_of_study,
                                                        satellite = satellite)
        
        if SGPS_v1:
                                                
            time_tag_v1 = SGPS_v1["L2_SciData_TimeStamp"]            
            time_tag_v1 = (time_tag_v1 * 1e9).astype(np.int64)
            dates_v1 = cdfepoch.to_datetime(time_tag_v1)
            
            flux_v1 = integrate_differential_flux(diff_proton_flux = SGPS_v1["AvgDiffProtonFlux"] * 1000,
                                                  int_proton_flux_over_500_MeV = SGPS_v1["AvgIntProtonFlux"],
                                                  diff_proton_lower_energy = SGPS_v1["DiffProtonLowerEnergy"] / 1000.0,
                                                  diff_proton_upper_energy = SGPS_v1["DiffProtonUpperEnergy"] / 1000.0,
                                                  diff_proton_effective_energy = SGPS_v1["DiffProtonEffectiveEnergy"] / 1000.0,
                                                  timestamps_per_file = SGPS_v1["timestamps_per_file"])
        if SGPS_v2:
            
            time_tag_v2 = SGPS_v2["L2_SciData_TimeStamp"]
            time_tag_v2 = (time_tag_v2 * 1e9).astype(np.int64)
            dates_v2 = cdfepoch.to_datetime(time_tag_v2)
            
            flux_v2 = integrate_differential_flux(diff_proton_flux = SGPS_v2["AvgDiffProtonFlux"] * 1000,
                                                  int_proton_flux_over_500_MeV = SGPS_v2["AvgIntProtonFlux"],
                                                  diff_proton_lower_energy = SGPS_v2["DiffProtonLowerEnergy"] / 1000.0,
                                                  diff_proton_upper_energy = SGPS_v2["DiffProtonUpperEnergy"] / 1000.0,
                                                  diff_proton_effective_energy = SGPS_v2["DiffProtonEffectiveEnergy"] / 1000.0,
                                                  timestamps_per_file = SGPS_v2["timestamps_per_file"])
            
        if SGPS_v3:
            
            time_tag_v3 = SGPS_v3["time"]            
            time_tag_v3 = (time_tag_v3 * 1e9).astype(np.int64)
            dates_v3 = cdfepoch.to_datetime(time_tag_v3)
            
            flux_v3 = integrate_differential_flux(diff_proton_flux = SGPS_v3["AvgDiffProtonFlux"] * 1000,
                                                  int_proton_flux_over_500_MeV = SGPS_v3["AvgIntProtonFlux"],
                                                  diff_proton_lower_energy = SGPS_v3["DiffProtonLowerEnergy"] / 1000.0,
                                                  diff_proton_upper_energy = SGPS_v3["DiffProtonUpperEnergy"] / 1000.0,
                                                  diff_proton_effective_energy = SGPS_v3["DiffProtonEffectiveEnergy"] / 1000.0,
                                                  timestamps_per_file = SGPS_v3["timestamps_per_file"])
            
            
        '''plt.semilogy(dates, flux)
        for start_date in start_dates:
            plt.vlines(x=start_date, ymin = 1, ymax = 20000, color="green", linewidth=7)
        for end_date in end_dates:
            plt.vlines(x=end_date, ymin = 1, ymax = 20000, color="red", linewidth=4)
        break'''
        
        break

print(periods_to_remove)

g08
g09
g10
g11
g13
g14
g15
g16
g17
g18
[]


C:\Users\brofe\AppData\Local\Temp\ipykernel_24312\1532985150.py:39: RuntimeWarning: Mean of empty slice
  integrated_flux.extend(np.nanmean([sensor_1_integral_flux_above_10_MeV, sensor_2_integral_flux_above_10_MeV], axis=0))


In [41]:
plt.semilogy(dates_v3, flux_v3)
